In [1]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=36e4f1d27cec89dadbea90b1c06bfa3fe451db34659c1f99e34ef39c4a2fe9da
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
import pandas as pd

train = pd.read_csv('/kaggle/input/15pp-triplets-dataset/train_15pp_triplets_dataset.csv')
test =  pd.read_csv('/kaggle/input/15pp-triplets-dataset/test_15pp_triplets_dataset.csv')

train.head(3)

,anchor,pos,neg,anchor_label,neg_label
0,A continuación se presenta un chat entre un cl...,A continuación se presenta un chat entre un cl...,A continuación se presenta un chat entre un cl...,RappiPrime,Account User
1,A continuación se presenta un chat entre un cl...,A continuación se presenta un chat entre un cl...,A continuación se presenta un chat entre un cl...,RappiPrime,Problems with promotions
2,A continuación se presenta un chat entre un cl...,A continuación se presenta un chat entre un cl...,A continuación se presenta un chat entre un cl...,RappiPrime,Problems with promotions


## Generate the Triplets Dataset

In [3]:
import logging
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

In [4]:
#from tqdm import tqdm
#from tqdm.contrib.logging import logging_redirect_tqdm

#triplets = []
#with logging_redirect_tqdm():
#    for i, v in tqdm(data.iterrows(), total = len(data)):
    #    anchor = v.prompt_embedding_v2
   #     label = v.grouped_label
  #      n_samples = len(labels) - 1
 #       pos_samples = data[data.grouped_label == label].sample(n_samples).prompt_embedding_v2.tolist()
#
     #   for l in labels:
    #        if l != label:
   #             pos = pos_samples.pop()
  #              neg = data[data.grouped_label == l].sample().iloc[0].prompt_embedding_v2
 #               triplets.append({'anchor': anchor, 'pos': pos, 'neg': neg})

In [5]:
#triplets_df = pd.DataFrame(triplets)

In [6]:
#from sklearn.model_selection import train_test_split
#train, test = train_test_split(triplets_df, test_size=0.2, random_state=0)

In [7]:
from sentence_transformers import InputExample
from tqdm import tqdm
from tqdm.contrib.logging import logging_redirect_tqdm

train_examples = []
test_examples = []

with logging_redirect_tqdm():
    for i, v in tqdm(train.iterrows(), total=len(train)):
      train_examples.append(InputExample(texts=[v.anchor, v.pos, v.neg]))

    for i, v in tqdm(test.iterrows(), total=len(test)):
      test_examples.append(InputExample(texts=[v.anchor, v.pos, v.neg]))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
100%|██████████| 3000/3000 [00:00<00:00, 12760.84it/s]


In [8]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=14)
test_dataloader = DataLoader(test_examples, shuffle=True, batch_size=3)

## Load and Train the Sentence Transformer model

In [9]:
from sentence_transformers import SentenceTransformer, models

with logging_redirect_tqdm():
    model_id = "distilbert-base-uncased"
    bert = models.Transformer(model_id)
    pooler = models.Pooling(
            bert.get_word_embedding_dimension(),
            pooling_mode_mean_tokens=True)
    model = SentenceTransformer(modules=[bert, pooler])
    log.info(model)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Use pytorch device: cuda
SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)


In [10]:
from sentence_transformers import losses

train_loss = losses.TripletLoss(model=model)

In [11]:
num_epochs = 4

log.info(f'Num epochs: {num_epochs}')

warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data

In [12]:
from sentence_transformers import evaluation
import random

test = test.sample(frac=1, random_state=0)

evaluator = evaluation.TripletEvaluator(anchors=test[:1000]['anchor'].tolist(), 
                            positives=test[:1000]['pos'].tolist(), 
                            negatives=test[:1000]['neg'].tolist(),
                            name='evaluation_metrics',
                            write_csv=True)

In [13]:
with logging_redirect_tqdm():
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              epochs=num_epochs,
              warmup_steps=warmup_steps,
              show_progress_bar=True,
              evaluator=evaluator,
              evaluation_steps=500,
              output_path='/kaggle/working/models_distilled',
              save_best_model=True,
              checkpoint_path='/kaggle/working/runs',
              checkpoint_save_steps=2000,
              checkpoint_save_total_limit=10) 

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/6136 [00:00<?, ?it/s]

TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 0 after 500 steps:
Accuracy Cosine Distance:   	62.20
Accuracy Manhattan Distance:	63.10
Accuracy Euclidean Distance:	63.20

Save model to /kaggle/working/models_distilled
TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 0 after 1000 steps:
Accuracy Cosine Distance:   	75.20
Accuracy Manhattan Distance:	76.50
Accuracy Euclidean Distance:	76.60

Save model to /kaggle/working/models_distilled
TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 0 after 1500 steps:
Accuracy Cosine Distance:   	73.40
Accuracy Manhattan Distance:	72.00
Accuracy Euclidean Distance:	71.10

TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 0 after 2000 steps:
Accuracy Cosine Distance:   	79.50
Accuracy Manhattan Distance:	78.90
Accuracy Euclidean Distance:	79.40

Save model to /kaggle/working/models_distilled
Save model to /kaggle/working/runs/2000
Trip

Iteration:   0%|          | 0/6136 [00:00<?, ?it/s]

TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 1 after 500 steps:
Accuracy Cosine Distance:   	83.70
Accuracy Manhattan Distance:	82.30
Accuracy Euclidean Distance:	81.50

TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 1 after 1000 steps:
Accuracy Cosine Distance:   	84.60
Accuracy Manhattan Distance:	83.40
Accuracy Euclidean Distance:	83.60

TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 1 after 1500 steps:
Accuracy Cosine Distance:   	83.40
Accuracy Manhattan Distance:	84.90
Accuracy Euclidean Distance:	83.80

Save model to /kaggle/working/runs/8000
TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 1 after 2000 steps:
Accuracy Cosine Distance:   	81.80
Accuracy Manhattan Distance:	83.80
Accuracy Euclidean Distance:	83.50

TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 1 after 2500 steps:
Accuracy Cosine Distance:   	81.70
Accuracy Man

Iteration:   0%|          | 0/6136 [00:00<?, ?it/s]

TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 2 after 500 steps:
Accuracy Cosine Distance:   	82.60
Accuracy Manhattan Distance:	84.40
Accuracy Euclidean Distance:	84.70

TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 2 after 1000 steps:
Accuracy Cosine Distance:   	82.30
Accuracy Manhattan Distance:	84.10
Accuracy Euclidean Distance:	84.00

TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 2 after 1500 steps:
Accuracy Cosine Distance:   	80.80
Accuracy Manhattan Distance:	83.70
Accuracy Euclidean Distance:	82.40

Save model to /kaggle/working/runs/14000
TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 2 after 2000 steps:
Accuracy Cosine Distance:   	81.20
Accuracy Manhattan Distance:	83.80
Accuracy Euclidean Distance:	83.20

TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 2 after 2500 steps:
Accuracy Cosine Distance:   	80.40
Accuracy Ma

Iteration:   0%|          | 0/6136 [00:00<?, ?it/s]

TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 3 after 500 steps:
Accuracy Cosine Distance:   	79.00
Accuracy Manhattan Distance:	82.20
Accuracy Euclidean Distance:	81.90

TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 3 after 1000 steps:
Accuracy Cosine Distance:   	79.90
Accuracy Manhattan Distance:	80.40
Accuracy Euclidean Distance:	79.80

TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 3 after 1500 steps:
Accuracy Cosine Distance:   	78.60
Accuracy Manhattan Distance:	81.50
Accuracy Euclidean Distance:	81.00

Save model to /kaggle/working/runs/20000
TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 3 after 2000 steps:
Accuracy Cosine Distance:   	78.60
Accuracy Manhattan Distance:	81.40
Accuracy Euclidean Distance:	81.00

TripletEvaluator: Evaluating the model on evaluation_metrics dataset in epoch 3 after 2500 steps:
Accuracy Cosine Distance:   	79.00
Accuracy Ma

In [14]:
!zip -r models_distilled.zip /kaggle/working/models_distilled

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  adding: kaggle/working/models_distilled/ (stored 0%)
  adding: kaggle/working/models_distilled/config.json (deflated 44%)
  adding: kaggle/working/models_distilled/1_Pooling/ (stored 0%)
  adding: kaggle/working/models_distilled/1_Pooling/config.json (deflated 47%)
  adding: kaggle/working/models_distilled/vocab.txt (deflated 53%)
  adding: kaggle/working/models_distilled/modules.json (deflated 53%)
  adding: kaggle/working/models_distilled/tokenizer_config.json (deflated 43%)
  adding: kaggle/working/models_distilled/eval/ (stored 0%)
  adding: kaggle/working/models_distilled/eval/triplet_evaluation_evaluation_metrics_results.csv (deflated 68%)
  adding: kaggle/wor